In [68]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [69]:
from bs4 import BeautifulSoup
import time

In [70]:
import os
from urllib.request import urlretrieve

In [71]:
options = webdriver.ChromeOptions()

options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

chrome_driver = './chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(chrome_driver, options=options)

In [73]:
disease_list = []

img_folder = './disease_img'

if not os.path.isdir(img_folder):
    os.mkdir(img_folder)

count = 1

for i in range(1, 65):
    driver.get('https://www.amc.seoul.kr/asan/healthinfo/disease/diseaseList.do?pageIndex='+str(i)+'&searchKeyword=')
    time.sleep(2)

    li = driver.find_element_by_class_name('descBox').find_elements_by_tag_name('li')

    size = len(li)
    for j in range(0, size):
        li = driver.find_element_by_class_name('descBox').find_elements_by_tag_name('li')
        li[j].find_element_by_tag_name('a').click()
        time.sleep(2)

        disease = {}

        img_src = driver.find_element_by_css_selector('.descBox > li > div.imgBox > img').get_attribute('src')
        urlretrieve(img_src, img_folder+'/'+str(count)+'.jpg')
        
        disease['name'] = driver.find_element_by_class_name('contTitle').text        
        
        dt = driver.find_elements_by_tag_name('dt')
        dd = driver.find_elements_by_tag_name('dd')
        
        for i in range(0, len(dd)):
            dt_text = dt[i].text
            
            if dt_text == '증상':
                aList = dd[i].find_elements_by_tag_name('a')
                if len(aList) > 0 :
                    symptoms = []                
                    for s in dd[i].find_elements_by_tag_name('a'):
                        symptoms.append(s.text)
                    disease['symptoms'] = symptoms
                else:
                    disease['symptom_description'] = dd[i].text
            elif dt_text == '관련질환':
                related = []
                for r in dd[i].find_elements_by_tag_name('a'):
                    related.append(r.text)
                disease['related'] = related
            elif dt_text == '진료과':
                subjects = []
                for s in dd[i].find_elements_by_tag_name('a'):
                    subjects.append(s.text)
                disease['subjects'] = subjects
            elif dt_text == '동의어':
                disease['synonym'] = dd[i].text
            elif dt_text =='정의':
                disease['definition'] = dd[i].text
            elif dt_text == '원인':
                disease['cause'] = dd[i].text
            elif dt_text == '진단':
                disease['diagnosis'] = dd[i].text
            elif dt_text == '치료':
                disease['treat'] = dd[i].text
            elif dt_text == '주의사항':
                disease['caution'] = dd[i].text
        driver.back()
        time.sleep(2)
        count += 1
        disease_list.append(disease)

In [74]:
import pandas as pd
df = pd.DataFrame()
for d in disease_list:
    df = df.append(d, ignore_index= True)

dis_df = dis_df.fillna('')
dis_df.to_csv('./disease.csv', index=False, encoding="utf-8-sig")